# 卷积神经网络
* 工作方式：特征提取+分类器（全连接）
    * 直接全连接会丧失原有的空间信息
    * 经过卷积层，通道数，宽，高都会变
    * 下采样：通道数不变，宽高减小，减少元素数量
    * 最终通过全连接，将三阶向量展开成一维向量
* 一般的图像是RGB形式，即三通道。
    * 三通道分别和各自的卷积核做卷积
    * 将算出来的三个卷积输出做加法，现在通道数变成1个
    * 多算几个卷积核，就可以得到多个输出通道

In [4]:
import torch
in_channels,out_channels = 5,10
width, height = 100,100
kernel_size = 3 #卷积核大小3*3,也可以设为(5,3)
batch_size = 1

inputs = torch.randn(batch_size,
                    in_channels,
                    width,
                    height)
conv_layer = torch.nn.Conv2d(in_channels,
                            out_channels,
                            kernel_size=kernel_size)

output = conv_layer(inputs)

print(inputs.shape)
print(output.shape)
print(conv_layer.weight.shape) #卷积层的参数：输入通道、输出通道、卷积核的大小

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 98, 98])
torch.Size([10, 5, 3, 3])


卷积层常见的参数
* padding：希望图像大小不变，在图像外面填几圈（3*3填1圈，最常见填0 3//2=1）
* stride: 步长，卷积时，重心走几步

In [10]:
import torch


inputs = [3,4,6,5,7,
         2,4,6,8,2,
         1,6,7,8,4,
          9,7,4,6,2,
         3,7,5,4,1]
inputs = torch.Tensor(inputs).view(1,1,5,5)

conv_layer = torch.nn.Conv2d(1,1,
                            kernel_size=3,
                            padding=1,
                            bias=False,stride=2)

kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3)
conv_layer.weight.data = kernel.data

output = conv_layer(inputs)

print(output)

tensor([[[[ 91., 224., 127.],
          [192., 282., 122.],
          [ 96., 110.,  31.]]]], grad_fn=<ConvolutionBackward0>)


下采样：
* MaxPooling：把图像分成2*2的小图像，求其中的最大值，只能针对一个通道做，所以最终通道数不变

In [11]:
import torch


inputs = [3,4,6,5,
         2,4,6,8,
         1,6,7,8,
          9,7,4,6]
inputs = torch.Tensor(inputs).view(1,1,4,4)

maxpooling_layer = torch.nn.MaxPool2d(kernel_size=2) #无权重

output = maxpooling_layer(inputs)

print(output)

tensor([[[[4., 8.],
          [9., 8.]]]])


In [16]:
# 实现一个简单的卷积神经网络
# 卷积层1*10*5*5->最大池化->10*20*5*5->最大池化->全连接

# MNIST数据集分类
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# 这里选了第一块gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# 1.数据准备
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,),(0.3081))]) #通道(0.1307,)均值，通道(0.3081)标准差
train_dataset = torchvision.datasets.MNIST(root = "./dataset/mnist", train=True, download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataset = torchvision.datasets.MNIST(root = "./dataset/mnist", train=False, download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

# 构造模型
# 2. 构造模型
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__() #调用父类的构造，必须要有
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(kernel_size=2)
        self.fc = torch.nn.Linear(320,10)
        
    def forward(self,x):
        batch_size = x.size(0) #有多少数据
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size,-1) 
        x = self.fc(x)
        return x

model = Model()

# 把模型中的东西都迁移到gpu上
model.to(device) 


# 3. 构造损失函数和优化器
# 这里损失函数用BCE   
criterion = torch.nn.CrossEntropyLoss() #对于输入z，做softmax,Log,-YlogY

# optim中有一个类叫SGD torch.optim.SGD() weight_decay(加一个w^Tw的优化目标)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)#告诉优化器对哪些Tensor做梯度优化，由model中的paramenters告知

# 4. 训练周期
# 把一轮epoch封装成一个函数
def train(epoch):
    running_loss = 0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        inputs,target = inputs.to(device), target.to(device)



        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
        
def test():
    correct = 0
    total = 0
    with torch.no_grad(): #主要是用于停止autograd模块的工作,以起到加速和节省显存的作用
        for data in test_loader:
            images,labels = data
            images,labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs.data,dim=1) #返回最大值及其索引
            total +=labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy on test set:{100*correct/total}%")
    
for epoch in range(10):
    train(epoch)
    test()

Accuracy on test set:96.34%
Accuracy on test set:97.68%
Accuracy on test set:97.86%
Accuracy on test set:98.36%
Accuracy on test set:98.6%
Accuracy on test set:98.57%
Accuracy on test set:98.75%
Accuracy on test set:98.64%
Accuracy on test set:98.88%
Accuracy on test set:98.53%


将代码迁移到GPU上运行
* 模型迁移
* 把用来计算的数据迁移，一定要和模型放到同一块显卡上



## 高级卷积神经网络
* 减少代码冗余：函数/类
* GoogleNet
    * 有一个6个卷积和1个池化组成的快，叫Inception
    * 神经网络的超参：卷积核的大小。把几种卷积核都用一下，将结果相加
    * 走不同路径的卷积(batch,channel,w,h)，channel可以变，但是w、h不能变。为了保持w、h不变，需要使用padding。
* 1x1卷积
    * 将原来张量中的值，挨个和这个卷积相乘
    * 作用：将不同channel相同位置上的信息进行融合。改变通道的数量，从而降低运算代价
* concatenate：w,h一样，所有的卷积输出，沿着通道的维度，拼接到一起

(batch,channel,w,h)

outputs = [branch1x1,branch5x5,branch3x3,branch_pool]
return torch.cat(outputs,dim=1) #因为沿着channel拼接到一起，所以dim=1

* 训练的时候，如果性能有提升，可以把当前网络的参数保存下来

In [27]:
import torch
class InceptionA(torch.nn.Module):
    def __init__(self,in_channels):
        super(InceptionA,self).__init__() #调用父类的构造，必须要有
        self.branch1x1 = torch.nn.Conv2d(in_channels,16,kernel_size=1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels,16,kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16,24,kernel_size=5,padding=2)
        
        self.branch3x3_1 = torch.nn.Conv2d(in_channels,16,kernel_size=1)
        self.branch3x3_2 = torch.nn.Conv2d(16,24,kernel_size=3,padding=1)
        self.branch3x3_3 = torch.nn.Conv2d(24,24,kernel_size=3,padding=1)
        
        self.branch_pool = torch.nn.Conv2d(in_channels,24,kernel_size=1)
        
    def forward(self,x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1,branch5x5,branch3x3,branch_pool]
        
        return torch.cat(outputs,dim=1)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__() #调用父类的构造，必须要有
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88,20,kernel_size=5)
        
        self.incep1 = InceptionA(in_channels=10)
        self.incep2 = InceptionA(in_channels=20)
        
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(1408,10)
        
    def forward(self,x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x))) # 10
        x = self.incep1(x) # 24*3+16=88
        x = F.relu(self.mp(self.conv2(x))) # 20
        x = self.incep2(x) # 24*3+16=88
        
        x = x.view(in_size,-1)
        
        x = self.fc(x)
        return x



In [28]:
# MNIST数据集分类
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# 这里选了第一块gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# 1.数据准备
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,),(0.3081))]) #通道(0.1307,)均值，通道(0.3081)标准差
train_dataset = torchvision.datasets.MNIST(root = "./dataset/mnist", train=True, download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataset = torchvision.datasets.MNIST(root = "./dataset/mnist", train=False, download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

model = Model()


# 3. 构造损失函数和优化器
# 这里损失函数用BCE   
criterion = torch.nn.CrossEntropyLoss() #对于输入z，做softmax,Log,-YlogY

# optim中有一个类叫SGD torch.optim.SGD() weight_decay(加一个w^Tw的优化目标)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)#告诉优化器对哪些Tensor做梯度优化，由model中的paramenters告知

# 4. 训练周期
# 把一轮epoch封装成一个函数
def train(epoch):
    running_loss = 0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        inputs,target = inputs.to(device), target.to(device)



        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
        
def test():
    correct = 0
    total = 0
    with torch.no_grad(): #主要是用于停止autograd模块的工作,以起到加速和节省显存的作用
        for data in test_loader:
            images,labels = data
            images,labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs.data,dim=1) #返回最大值及其索引
            total +=labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy on test set:{100*correct/total}%")
    
for epoch in range(10):
    train(epoch)
    test()

Accuracy on test set:96.57%
Accuracy on test set:97.45%
Accuracy on test set:98.09%
Accuracy on test set:98.3%
Accuracy on test set:98.4%
Accuracy on test set:98.48%
Accuracy on test set:98.55%
Accuracy on test set:98.49%
Accuracy on test set:98.84%
Accuracy on test set:98.73%


## ResNet
* 梯度消失：导致离输入较近的权重没有办法得到训练
* 加一个跳连接 ResidualBlock

In [33]:
import torch
class ResidualBlock(torch.nn.Module):
    def __init__(self,channels):
        super(ResidualBlock,self).__init__() #调用父类的构造，必须要有
        self.channels = channels #输出和输入的通道数要一样
        self.conv1 = torch.nn.Conv2d(channels,channels,kernel_size=3,padding=1)
        self.conv2 = torch.nn.Conv2d(channels,channels,kernel_size=3,padding=1)
        
    def forward(self,x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)

        return F.relu(x+y)
    
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__() #调用父类的构造，必须要有
        self.conv1 = torch.nn.Conv2d(1,16,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(16,32,kernel_size=5)
        self.mp = torch.nn.MaxPool2d(2)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        
        self.fc = torch.nn.Linear(512,10)
        
    def forward(self,x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        
        x = x.view(in_size,-1)
        
        x = self.fc(x)
        return x

In [34]:
# MNIST数据集分类
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# 这里选了第一块gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# 1.数据准备
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,),(0.3081))]) #通道(0.1307,)均值，通道(0.3081)标准差
train_dataset = torchvision.datasets.MNIST(root = "./dataset/mnist", train=True, download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataset = torchvision.datasets.MNIST(root = "./dataset/mnist", train=False, download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

model = Model()


# 3. 构造损失函数和优化器
# 这里损失函数用BCE   
criterion = torch.nn.CrossEntropyLoss() #对于输入z，做softmax,Log,-YlogY

# optim中有一个类叫SGD torch.optim.SGD() weight_decay(加一个w^Tw的优化目标)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)#告诉优化器对哪些Tensor做梯度优化，由model中的paramenters告知

# 4. 训练周期
# 把一轮epoch封装成一个函数
def train(epoch):
    running_loss = 0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        inputs,target = inputs.to(device), target.to(device)



        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
        
def test():
    correct = 0
    total = 0
    with torch.no_grad(): #主要是用于停止autograd模块的工作,以起到加速和节省显存的作用
        for data in test_loader:
            images,labels = data
            images,labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs.data,dim=1) #返回最大值及其索引
            total +=labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy on test set:{100*correct/total}%")
    
for epoch in range(10):
    train(epoch)
    test()

Accuracy on test set:97.21%
Accuracy on test set:98.15%
Accuracy on test set:98.58%
Accuracy on test set:98.64%
Accuracy on test set:98.74%
Accuracy on test set:98.57%
Accuracy on test set:98.94%
Accuracy on test set:98.93%
Accuracy on test set:98.96%
Accuracy on test set:98.99%


# 建议
1. 理论《深度学习》
2. 阅读PyTorch文档（通读一遍）
3. 复现经典工作
    * 读代码
    * 尝试自己写
4. 扩充视野：知识+写代码